In [1]:
import json
import os
os.environ["USE_SCHEMA_BATCH_MODE"] = "True"
from Schema import ExtractedData
import pandas as pd

Schema: Batch Mode


In [2]:
with open("prompt.json", "r") as f:
    prompts = json.load(f)
    base_prompt = prompts["base_prompt"]
    system_prompt = prompts["system_prompt"]

print("System Prompt:", system_prompt)
print("Prompt:", base_prompt)

System Prompt: You are an expert in extracting information from recepies.
Prompt: Read the recipe and fill out the ExtractedData accordingly. Also give the recipe a funny pretentious name.



In [3]:
MAX_TOKENS = 2000 # Adjust as needed
def make_request(custom_id, system_prompt, user_prompt, response_format):
    tmp = {"custom_id": custom_id, "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-batch-2", "messages": [{"role": "system", "content": system_prompt},{"role": "user", "content": user_prompt}],"max_tokens": MAX_TOKENS, "response_format": "PLACEHOLDER", "temperature": 0.2}}
    tmp = json.dumps(tmp, ensure_ascii=False)
    tmp = tmp.replace('"PLACEHOLDER"', response_format)
    return tmp

In [4]:
# you can have up to 50k requests in one file and files may be up to 200MB in size
SPLITS = 2 # adjust as needed
DATASET = "[NAME]_whatever" # adjust as needed

df = pd.read_csv("data.csv")
split_points = list(range(len(df)))[::len(df)//SPLITS]

schema = ExtractedData.model_json_schema()
schema = json.dumps(schema, ensure_ascii=False)

os.makedirs("files_jsonl", exist_ok=True)
for split_idx in range(SPLITS):

    start_idx = split_points[split_idx]
    end_idx = split_points[split_idx + 1] if split_idx + 1 < SPLITS else len(df)

    with open(f'files_jsonl/{DATASET}-{start_idx}-{end_idx}.jsonl', 'w') as f:
        for index, row in list(df.iterrows())[start_idx:end_idx]:
            b_id = str(row["id"])
            prompt = base_prompt + row["text"]
            request = make_request(b_id, system_prompt, prompt, schema)
            f.write(request + "\n")